# 第二章 字符串和文本

## 但是，一些更为复杂的操作可能需要正则表达式或者强大的解析器，所有这些主题我们都会详细讲解。并且在操作Unicode时候碰到的一些棘手的问题在这里也会被提及到。

## 2.1 使用多个界定符分割字符串

你需要将一个字符串分割为多个字段，但是分隔符(还有周围的空格) 并不是固定
的。

In [2]:
line = 'asdf fjdk; afed, fjek,asdf, foo'

In [3]:
import re

In [5]:
re.split(r'[;,\s]\s*',line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

## 2.2 字符串开头或结尾匹配

检查字符串开头或结尾的一个简单方法是使用str.startswith() 或者是
str.endswith() 方法。比如：

In [6]:
filename = 'sample.txt'
filename.endswith('.txt')

True

In [7]:
filename = [ 'Makefile', 'foo.c', 'bar.py', 'spam.c', 'spam.h' ]

In [8]:
[f for f in filename if f.endswith(('.c','.h'))]

['foo.c', 'spam.c', 'spam.h']

<b>奇怪的是，这个方法中必须要输入一个元组作为参数。如果你恰巧有一个list 或
者set 类型的选择项，要确保传递参数前先调用tuple() 将其转换为元组类型。比如：</b>

In [9]:
choices = ['http:', 'ftp:']
url = 'http://www.python.org'

In [10]:
url.endswith(choices)

TypeError: endswith first arg must be str or a tuple of str, not list

In [12]:
url.startswith(tuple(choices))

True

## 2.3 用Shell 通配符匹配字符串

fnmatch 模块提供了两个函数—— fnmatch() 和fnmatchcase() ，可以用来实现
这样的匹配。用法如下：

In [14]:
from fnmatch import fnmatch,fnmatchcase

In [15]:
fnmatch('foo.txt','*.txt')

True

In [16]:
fnmatch('foo.txt','*.TXT')

True

In [17]:
fnmatchcase('foo.txt','*.TXT')

False

## 2.4 字符串匹配和搜索

In [18]:
text = 'yeah, but no, but yeah, but no, but yeah'

In [19]:
text.find('no')

10

In [20]:
text1 = '11/27/2012'

In [24]:
if re.match(r'\d+/\d+/\d+',text1):
    print('YES')
else:
    print('NO')

YES


match() 总是从字符串开始去匹配，如果你想查找字符串任意部分的模式出现位
置，使用findall() 方法去代替。比如：

In [25]:
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'

In [26]:
re.findall(r'\d+/\d+/\d+',text)

['11/27/2012', '3/13/2013']

在定义正则式的时候，通常会利用括号去捕获分组。

In [27]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')

In [28]:
m = datepat.match('11/27/2012')

In [29]:
m.groups()

('11', '27', '2012')

In [30]:
datepat.findall(text)

[('11', '27', '2012'), ('3', '13', '2013')]

In [31]:
for month,day,year in datepat.findall(text):
    print('{}-{}-{}'.format(year,month,day))

2012-11-27
2013-3-13


## 2.5 字符串搜索和替换

对于简单的字面模式，直接使用str.repalce() 方法即可，比如：

In [32]:
text = 'yeah, but no, but yeah, but no, but yeah'

In [33]:
text.replace('yeah','yep')

'yep, but no, but yep, but no, but yep'

对于复杂的模式，请使用re 模块中的sub() 函数。为了说明这个，假设你想将形
式为11/27/2012 的日期字符串改成2012-11-27 。示例如下：

In [34]:
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'

In [36]:
re.sub(r'(\d+)/(\d+)/(\d+)',r'\3-\1-2',text)

'Today is 2012-11-2. PyCon starts 2013-3-2.'

In [38]:
datepat.sub(r'\3-\1-\2',text)

'Today is 2012-11-27. PyCon starts 2013-3-13.'

对于更加复杂的替换，可以传递一个替换回调函数来代替，比如：

In [40]:
from calendar import month_abbr
def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2),mon_name,m.group(3))
datepat.sub(change_date,text)

'Today is 27 Nov 2012. PyCon starts 13 Mar 2013.'

如果除了替换后的结果外，你还想知道有多少替换发生了，可以使用re.subn()
来代替。

In [42]:
new_text,n = datepat.subn(change_date,text)

In [43]:
n

2

## 2.6 字符串忽略大小写的搜索替换

为了在文本操作时忽略大小写，你需要在使用re 模块的时候给这些操作提供
re.IGNORECASE 标志参数。

In [45]:
text = 'UPPER PYTHON, lower python, Mixed Python'

In [46]:
re.findall('python',text,flags=re.IGNORECASE)

['PYTHON', 'python', 'Python']

In [47]:
re.findall('python',text,flags=re.I)

['PYTHON', 'python', 'Python']

## 2.7 最短匹配模式

你正在试着用正则表达式匹配某个文本模式，但是它找到的是模式的最长可能匹
配。而你想修改它变成查找最短的可能匹配。

In [1]:
import re

In [3]:
str_pat = re.compile(r'\"(.*)\"')
text1 = 'Computer says "no."'
str_pat.findall(text1)

['no.']

In [4]:
text2 = 'Computer says "no." Phone says "yes."'
str_pat.findall(text2)

['no." Phone says "yes.']

<b>这里可以发现取多了</b>

在这个例子中，模式r'n"(.*)n"' 的意图是匹配被双引号包含的文本。但是在正
则表达式中* 操作符是贪婪的，因此匹配操作会查找最长的可能匹配。于是在第二个
例子中搜索text2 的时候返回结果并不是我们想要的。为了修正这个问题，可以在模式中的* 操作符后面加上? 修饰符，就像这样：

In [5]:
str_pat = re.compile(r'\"(.*?)\"')
str_pat.findall(text2)

['no.', 'yes.']

## 2.9 删除字符串中不需要的字符
### 你想去掉文本字符串开头，结尾或者中间不想要的字符，比如空白。

In [6]:
s = ' hello world \n'
s.strip()

'hello world'

In [7]:
s.lstrip()

'hello world \n'

In [8]:
s.rstrip()

' hello world'

In [9]:
t = '-----hello====='

In [10]:
t.lstrip('-')

'hello====='

In [11]:
t.strip('-=')

'hello'

但是需要注意的是去除操作不会对字符串的中间的文本产生任何影响。比如：

In [15]:
s = ' hello          world \n'
s = s.strip()

如果你想处理中间的空格，那么你需要求助其他技术。比如使用replace() 方法
或者是用正则表达式替换。示例如下：

In [16]:
s.replace(' ','')

'helloworld'

In [17]:
s

'hello          world'

In [19]:
re.sub(r'\s+','',s)

'helloworld'

## 2.12 审查清理文本字符串

### 一些无聊的幼稚黑客在你的网站页面表单中输入文本”pýtĥöñ”，然后你想将这些字符清理掉。

In [20]:
s = 'pýtĥöñ\fis\tawesome\r\n'

In [21]:
s

'pýtĥöñ\x0cis\tawesome\r\n'

第一步是清理空白字符。为了这样做， 先创建一个小的转换表格然后使用
translate() 方法：

In [23]:
remap ={
    ord('\t'):' ',
    ord('\f'):' ',
    ord('\r'):None,
}
s.translate(remap)

'pýtĥöñ is awesome\n'

你可以以这个表格为基础进一步构建更大的表格。比如，让我们删除所有的和音
符：

In [24]:
import unicodedata
import sys
cb_chrs = dict.fromkeys(c for c in range(sys.maxunicode)
                       if unicodedata.combining(chr(c)))

In [28]:
s = s.translate(remap)
b = unicodedata.normalize('NFD',s)
b

'pýtĥöñ is awesome\n'

In [29]:
b.translate(cb_chrs)

'python is awesome\n'

In [30]:
s

'pýtĥöñ is awesome\n'

In [31]:
b = unicodedata.normalize('NFD',s)

In [32]:
b.encode('ascii','ignore').decode('ascii')

'python is awesome\n'

## 2.13 字符串对齐

##  你想通过某种对齐方式来格式化字符串

In [33]:
text = 'Hello World'

In [34]:
text.ljust(20)

'Hello World         '

In [35]:
text.rjust(20)

'         Hello World'

In [36]:
text.ljust(20,'=')

'Hello World========='

In [37]:
text.ljust(20,'*')

'Hello World*********'

In [38]:
format(text,'>20')

'         Hello World'

format(text,'^20')

In [44]:
format(text,':^20s')

'::::Hello World:::::'

In [45]:
'{:>10s} {:>10s}'.format('Hello', 'World')

'     Hello      World'

In [46]:
'{:>10s} {:>10s}'.format('Hello', 'World')

'     Hello      World'

## 2.14 合并拼接字符串

如果你想要合并的字符串是在一个序列或者iterable 中，那么最快的方式就是使
用join() 方法。比如：

In [48]:
parts = ['Is', 'Chicago', 'Not', 'Chicago?']

In [49]:
' '.join(parts)

'Is Chicago Not Chicago?'

In [51]:
print('a', 'b', 'c', sep=':')

a:b:c


## 2.15 字符串中插入变量

In [52]:
s = '{name} has {n} messages.'

In [53]:
s.format(name='hadxu',n=4)

'hadxu has 4 messages.'

In [54]:
s.format('hadxu',4)

KeyError: 'name'

In [57]:
name = 'hadxu'
n = 4
s.format_map(vars())

'hadxu has 4 messages.'

## 2.17 在字符串中处理html 和xml

In [58]:
s = 'Elements are written as "<tag>text</tag>".'

In [59]:
s

'Elements are written as "<tag>text</tag>".'

In [60]:
import html

In [61]:
html.escape(s)

'Elements are written as &quot;&lt;tag&gt;text&lt;/tag&gt;&quot;.'

In [62]:
html.escape(s,quote=False)

'Elements are written as "&lt;tag&gt;text&lt;/tag&gt;".'

## 2.18 字符串令牌解析

## 下面的cookbook问题都比较困难，我先放着，等需要了解的时候过来看
### 关于令牌解析，递归下降解析器